Парсер sudact.ru

Часть вторая: собирает дела по списку ссылок.

Скрипт примитивен и требует постоянного контроля.

Сам sudact парсится легко, но выдает капчу примерно через каждые 200 запросов. Увеличение задержки не помогает.

Последние правки скрипта: 2022-01-31

## Библиотеки и настройки

In [1]:
#Загружаю нужные библиотеки

import pandas as pd
import math
import os

import re

# для юзер-агента
from fake_useragent import UserAgent

# прогресс-бар
from tqdm.notebook import trange, tqdm

# для пауз при парсинге
from time import sleep

# селениум
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service
from selenium import webdriver

from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
# настройки юзер-агента

ua = UserAgent()

# юзер-агент для Хрома
user_agent = ua.chrome

In [3]:
test_url = 'https://intoli.com/blog/not-possible-to-block-chrome-headless/chrome-headless-test.html'

In [4]:
# путь к файлам

file_path = 'C:/00_Data/' + input() + '/'
proxy_list_path = 'C:/work/keys/proxy_list.txt'

court


In [5]:
# загружаю список прокси
# список простой текстовый файл, который содержит прокси в формате: адрес:порт логин:пароль

proxy_list = []

with open(proxy_list_path, 'r') as f:
    for line in f:
        proxy_list += [line.split()]
        
print('Рабочих прокси — ' + str(len(proxy_list)))

Рабочих прокси — 9


In [6]:
# номер прокси проще прописать вручную, ввод со строки постоянно глючит

print('Номер прокси, от 0 до ' + str(len(proxy_list) - 1))
proxy_number = int(input())

Номер прокси, от 0 до 8
6


In [7]:
# настройки веб-драйвера, чтобы скрыть автоматическое управление

opt = webdriver.ChromeOptions()
opt.add_argument(f'user-agent={user_agent}')
opt.add_argument('--disable-blink-features')
opt.add_argument('--disable-blink-features=AutomationControlled')
opt.add_argument(f'--proxy-server={proxy_list[proxy_number][0]}') # "девственный" прокси(:
opt.add_experimental_option("excludeSwitches", ["enable-automation"]) # отключает строку про удаленное управление
opt.add_experimental_option('useAutomationExtension', False)

In [8]:
print('Login: ' + proxy_list[proxy_number][1])
print('Pass: ' + proxy_list[proxy_number][2])

Login: tw1QwuaQ
Pass: EJ1SMvdd


In [9]:
# запускаю драйвер, проверяю настройки

#driver = Chrome('C:/Users/bgi/Anaconda3/chromedriver/chromedriver', options=opt)

driver = webdriver.Chrome(options=opt)
driver.get(test_url)
driver.maximize_window() # раскрываю окно на максимум

## Собираю дела

#### Загружаю ссылки

In [16]:
work_urls = pd.read_excel(file_path + 'new_11.02.2022.xlsx')

In [17]:
work_urls.shape

(1978, 3)

In [18]:
url_list = work_urls.act_url.to_list()

In [19]:
len(url_list)

1978

### Код парсера

In [14]:
data = [] # список для данных

In [21]:
for url in tqdm(url_list[1800:]):
    
    driver.get(url)
    sleep(3)
    
    title = driver.find_element(By.TAG_NAME, 'h1').text # заголовок
    
    court_info = driver.find_element(By.CLASS_NAME, 'b-justice').text # инфо о суде и деле
        
    act_text = driver.find_element(By.CLASS_NAME, 'h-col1').text # текст дела
    
    # упоминание статей
    articles = []
    
    article_hrefs = driver.find_elements(By.PARTIAL_LINK_TEXT, 'РФ')
    
    for item in article_hrefs:
        articles += [item.text]
        
    # html код блока     
    html_text = driver.find_element(By.CLASS_NAME, 'h-col1').get_attribute('innerHTML')
           
        
    data.append([title, court_info, act_text, url, articles, html_text])

  0%|          | 0/178 [00:00<?, ?it/s]

In [22]:
# записываю в датафрейм

head_list = ['act_name', 'court_info', 'act_text', 'act_url', 'articles', 'html_text']

work_texts = pd.DataFrame(data, columns=head_list)

In [23]:
work_texts.sample(5)

,act_name,court_info,act_text,act_url,articles,html_text
263,Постановление № 5-213/2020 от 22 июля 2020 г. ...,Сосновский районный суд (Челябинская область) ...,Постановление № 5-213/2020 от 22 июля 2020 г. ...,https://sudact.ru/regular/doc/0tXFCzNtXob0/,[Судебные и нормативные акты РФ],"<div class=""bookmarkBlockContainer""><div class..."
288,Приговор № 1-137/2020 от 29 сентября 2020 г. п...,Волосовский районный суд (Ленинградская област...,Приговор № 1-137/2020 от 29 сентября 2020 г. п...,https://sudact.ru/regular/doc/UdkqhDhVRtQS/,"[Судебные и нормативные акты РФ, 325 УК РФ, 16...","<div class=""bookmarkBlockContainer""><div class..."
196,Решение № 2А-429/2020 2А-429/2020~М-414/2020 М...,Некрасовский районный суд (Ярославская область...,Решение № 2А-429/2020 2А-429/2020~М-414/2020 М...,https://sudact.ru/regular/doc/yQqDBc8hJ7Lm/,"[Судебные и нормативные акты РФ, 39 КАС РФ, 32...","<div class=""bookmarkBlockContainer""><div class..."
476,Решение № 2А-429/2020 2А-429/2020~М-414/2020 М...,Некрасовский районный суд (Ярославская область...,Решение № 2А-429/2020 2А-429/2020~М-414/2020 М...,https://sudact.ru/regular/doc/yQqDBc8hJ7Lm/,"[Судебные и нормативные акты РФ, 39 КАС РФ, 32...","<div class=""bookmarkBlockContainer""><div class..."
207,Приговор № 1-361/2020 от 3 ноября 2020 г. по д...,Мичуринский городской суд (Тамбовская область)...,Приговор № 1-361/2020 от 3 ноября 2020 г. по д...,https://sudact.ru/regular/doc/Hki8tboIHGgA/,"[Судебные и нормативные акты РФ, 322.1 УК РФ, ...","<div class=""bookmarkBlockContainer""><div class..."


In [24]:
work_texts.shape

(779, 6)

In [25]:
# номер файла

number = 3

In [26]:
work_texts.to_csv(file_path + f'new_{number}.csv', encoding='utf8', index=False)

In [27]:
driver.close()

#### Объединяю с урлами

In [ ]:
work_urls.head()

In [ ]:
work_texts.head()

In [ ]:
work_texts.shape

In [ ]:
work_texts = work_texts.merge(work_urls, on=['act_name', 'act_url'], how='left')

In [ ]:
work_texts.shape

In [ ]:
#work_texts.to_csv(file_path + 'new_bears.csv', encoding='utf8', index=False)

#### Добавляю запросы в датасет

*Если они есть*

In [ ]:
urls_original = pd.read_excel('C:/00_Projects/RIA/2022/02_elk_accidents/00_data/sudact/urls_elk.xlsx', sheet_name='original')

In [ ]:
urls_original.head(3)

In [ ]:

only_urls = (urls_original.groupby('act_url', as_index=False)
 .agg({'instance': 'count'}))

In [ ]:
only_urls

In [ ]:
def apply_queries(df):
    
     return urls_original[urls_original['act_url'] == df.act_url]['need_text'].to_list()

In [ ]:
only_urls['queries'] = only_urls.apply(apply_queries, axis=1)

In [ ]:
only_urls = only_urls[['act_url', 'queries']]

In [ ]:
work_texts = work_texts.merge(only_urls, how='left', on='act_url')

In [ ]:
work_texts

#### Осмысленные значения инстанции

In [ ]:
# осмысленные значения инстанции

work_texts['instance'] = work_texts['instance'].replace(10, 'первая', regex=True)
work_texts['instance'] = work_texts['instance'].replace(20, 'аппеляция', regex=True)
work_texts['instance'] = work_texts['instance'].replace(30, 'кассация', regex=True)
work_texts['instance'] = work_texts['instance'].replace(40, 'пересмотр', regex=True)
work_texts['instance'] = work_texts['instance'].replace(60, 'надзор', regex=True)

In [ ]:
work_texts.to_csv(file_path + 'new_243.csv', encoding='utf8', index=False)